<a href="https://colab.research.google.com/github/yunpengliDataScience/Skin_Cancer_ML_DL/blob/main/Skin_Cancer_Analysis/Generate_csv_for_DeepLearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import sys
import os
from imageio import imread
from PIL import Image
from glob import glob
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from sklearn import preprocessing

In [38]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [39]:
project_root_path_string = '/content/gdrive/My Drive/Data606/Project/Skin_Cancer_Analysis/'

print("project_root_path_string:", project_root_path_string)

project_root_path_string: /content/gdrive/My Drive/Data606/Project/Skin_Cancer_Analysis/


In [ ]:
sys.path.append(project_root_path_string)

sys.path

['',
 '/content',
 '/env/python',
 '/usr/lib/python37.zip',
 '/usr/lib/python3.7',
 '/usr/lib/python3.7/lib-dynload',
 '/usr/local/lib/python3.7/dist-packages',
 '/usr/lib/python3/dist-packages',
 '/usr/local/lib/python3.7/dist-packages/IPython/extensions',
 '/root/.ipython',
 '/content/gdrive/My Drive/Data606/Project/Skin_Cancer_Analysis/']

In [ ]:
data_root_path = os.path.join(project_root_path_string, 'data') 

print("data_root_path:", data_root_path)

data_root_path: /content/gdrive/My Drive/Data606/Project/Skin_Cancer_Analysis/data


In [ ]:
raw_data_path =  os.path.join(data_root_path, 'raw_image_data') 
print("raw_data_path:", raw_data_path)

raw_data_path: /content/gdrive/My Drive/Data606/Project/Skin_Cancer_Analysis/data/raw_image_data


In [ ]:
tabular_data_path = os.path.join(data_root_path, 'tabular_data') 

print("tabular_data_path:", tabular_data_path)

tabular_data_path: /content/gdrive/My Drive/Data606/Project/Skin_Cancer_Analysis/data/tabular_data


In [ ]:
metaData_pd = pd.read_csv(os.path.join(tabular_data_path, 'metadata.csv'))

In [ ]:
metaData_pd.head()

,patient_id,lesion_id,smoke,drink,background_father,background_mother,age,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,has_sewage_system,fitspatrick,region,diameter_1,diameter_2,diagnostic,itch,grew,hurt,changed,bleed,elevation,img_id,biopsed
0,PAT_1516,1765,NaN,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ARM,NaN,NaN,NEV,False,False,False,False,False,False,PAT_1516_1765_530.png,False
1,PAT_46,881,False,False,POMERANIA,POMERANIA,55,False,FEMALE,True,True,True,True,3.0,NECK,6.0,5.0,BCC,True,True,False,True,True,True,PAT_46_881_939.png,True
2,PAT_1545,1867,NaN,NaN,NaN,NaN,77,NaN,NaN,NaN,NaN,NaN,NaN,NaN,FACE,NaN,NaN,ACK,True,False,False,False,False,False,PAT_1545_1867_547.png,False
3,PAT_1989,4061,NaN,NaN,NaN,NaN,75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,HAND,NaN,NaN,ACK,True,False,False,False,False,False,PAT_1989_4061_934.png,False
4,PAT_684,1302,False,True,POMERANIA,POMERANIA,79,False,MALE,True,False,False,False,1.0,FOREARM,5.0,5.0,BCC,True,True,False,False,True,True,PAT_684_1302_588.png,True


In [ ]:
count_nan_in_df = metaData_pd.isnull().sum()
print (count_nan_in_df)

patient_id               0
lesion_id                0
smoke                  804
drink                  804
background_father      818
background_mother      822
age                      0
pesticide              804
gender                 804
skin_cancer_history    804
cancer_history         804
has_piped_water        804
has_sewage_system      804
fitspatrick            804
region                   0
diameter_1             804
diameter_2             804
diagnostic               0
itch                     0
grew                     0
hurt                     0
changed                  0
bleed                    0
elevation                0
img_id                   0
biopsed                  0
dtype: int64


In [ ]:
 le = preprocessing.LabelEncoder()

In [ ]:
# categorical features check
cat =metaData_pd.select_dtypes(include='O').keys()
cat

Index(['patient_id', 'smoke', 'drink', 'background_father',
       'background_mother', 'pesticide', 'gender', 'skin_cancer_history',
       'cancer_history', 'has_piped_water', 'has_sewage_system', 'region',
       'diagnostic', 'itch', 'grew', 'hurt', 'changed', 'bleed', 'elevation',
       'img_id'],
      dtype='object')

In [ ]:
# replace the missing for data categorical variable based on the mode function. most frequent value.
cat_columns = metaData_pd[['smoke', 'drink', 'background_father', 'background_mother', 'pesticide',
       'gender', 'skin_cancer_history', 'cancer_history', 'has_piped_water',
       'has_sewage_system', 'region', 'itch', 'grew', 'hurt',
       'changed', 'bleed', 'elevation','biopsed']]
# cat_columns=cat_columns.fillna(cat_columns.columns.mode())
# cat_columns
for column in cat_columns.columns:
    cat_columns[column].fillna(cat_columns[column].mode()[0], inplace=True)

/usr/local/lib/python3.7/dist-packages/pandas/core/series.py:4536: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  downcast=downcast,


In [ ]:
cat_columns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2298 entries, 0 to 2297
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   smoke                2298 non-null   bool  
 1   drink                2298 non-null   bool  
 2   background_father    2298 non-null   object
 3   background_mother    2298 non-null   object
 4   pesticide            2298 non-null   bool  
 5   gender               2298 non-null   object
 6   skin_cancer_history  2298 non-null   bool  
 7   cancer_history       2298 non-null   bool  
 8   has_piped_water      2298 non-null   bool  
 9   has_sewage_system    2298 non-null   bool  
 10  region               2298 non-null   object
 11  itch                 2298 non-null   object
 12  grew                 2298 non-null   object
 13  hurt                 2298 non-null   object
 14  changed              2298 non-null   object
 15  bleed                2298 non-null   object
 16  elevat

In [ ]:
metaData_le= cat_columns
##numeric features impute mean values for nan
#Taking care of Missing data
col=metaData_pd[['age','fitspatrick','diameter_1','diameter_2']]
from sklearn.impute import SimpleImputer  
imputer = SimpleImputer(missing_values=np.nan, strategy='mean')
imputer = imputer.fit(col)
###merge the numeric fields to the categorical fields
metaData_le[['age','fitspatrick','diameter_1','diameter_2']]= imputer.transform(col)

/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:659: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[k] = np.nan
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [ ]:
metaData_le.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2298 entries, 0 to 2297
Data columns (total 22 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   smoke                2298 non-null   bool   
 1   drink                2298 non-null   bool   
 2   background_father    2298 non-null   object 
 3   background_mother    2298 non-null   object 
 4   pesticide            2298 non-null   bool   
 5   gender               2298 non-null   object 
 6   skin_cancer_history  2298 non-null   bool   
 7   cancer_history       2298 non-null   bool   
 8   has_piped_water      2298 non-null   bool   
 9   has_sewage_system    2298 non-null   bool   
 10  region               2298 non-null   object 
 11  itch                 2298 non-null   object 
 12  grew                 2298 non-null   object 
 13  hurt                 2298 non-null   object 
 14  changed              2298 non-null   object 
 15  bleed                2298 non-null   o

In [ ]:
metaData_le=metaData_le.join(metaData_pd['diagnostic'])
metaData_le.head()

,smoke,drink,background_father,background_mother,pesticide,gender,skin_cancer_history,cancer_history,has_piped_water,has_sewage_system,region,itch,grew,hurt,changed,bleed,elevation,biopsed,age,fitspatrick,diameter_1,diameter_2,diagnostic
0,False,False,POMERANIA,POMERANIA,False,FEMALE,False,True,True,True,ARM,False,False,False,False,False,False,False,8.0,2.26573,11.897055,8.852209,NEV
1,False,False,POMERANIA,POMERANIA,False,FEMALE,True,True,True,True,NECK,True,True,False,True,True,True,True,55.0,3.00000,6.000000,5.000000,BCC
2,False,False,POMERANIA,POMERANIA,False,FEMALE,False,True,True,True,FACE,True,False,False,False,False,False,False,77.0,2.26573,11.897055,8.852209,ACK
3,False,False,POMERANIA,POMERANIA,False,FEMALE,False,True,True,True,HAND,True,False,False,False,False,False,False,75.0,2.26573,11.897055,8.852209,ACK
4,False,True,POMERANIA,POMERANIA,False,MALE,True,False,False,False,FOREARM,True,True,False,False,True,True,True,79.0,1.00000,5.000000,5.000000,BCC


In [ ]:
metaData_le.columns

Index(['smoke', 'drink', 'background_father', 'background_mother', 'pesticide',
       'gender', 'skin_cancer_history', 'cancer_history', 'has_piped_water',
       'has_sewage_system', 'region', 'itch', 'grew', 'hurt', 'changed',
       'bleed', 'elevation', 'biopsed', 'age', 'fitspatrick', 'diameter_1',
       'diameter_2', 'diagnostic'],
      dtype='object')

In [ ]:
# ordinal encoding
from sklearn.preprocessing import OrdinalEncoder
# define ordinal encoding
encoder = OrdinalEncoder()
# transform data
data_enc = encoder.fit_transform(metaData_le)
# y_enc= encoder.fit_transform(y2.dropna())

In [ ]:
np.unique(data_enc[:,-1])

array([0., 1., 2., 3., 4., 5.])

In [ ]:
# Changing Numpy array to dataframe
metaData_le = pd.DataFrame(data_enc, columns = ['smoke', 'drink', 'background_father', 'background_mother', 'pesticide',
       'gender', 'skin_cancer_history', 'cancer_history', 'has_piped_water',
       'has_sewage_system', 'region', 'itch', 'grew', 'hurt', 'changed',
       'bleed', 'elevation', 'biopsed', 'age', 'fitspatrick', 'diameter_1',
       'diameter_2', 'target'])

print(metaData_le)
print(type(metaData_le))

      smoke  drink  background_father  ...  diameter_1  diameter_2  target
0       0.0    0.0                9.0  ...        13.0        10.0     3.0
1       0.0    0.0                9.0  ...         7.0         6.0     1.0
2       0.0    0.0                9.0  ...        13.0        10.0     0.0
3       0.0    0.0                9.0  ...        13.0        10.0     0.0
4       0.0    1.0                9.0  ...         6.0         6.0     1.0
...     ...    ...                ...  ...         ...         ...     ...
2293    0.0    0.0                9.0  ...        13.0        10.0     0.0
2294    0.0    0.0                9.0  ...        15.0        14.0     1.0
2295    0.0    0.0                9.0  ...        13.0        10.0     5.0
2296    0.0    0.0                9.0  ...         6.0         5.0     1.0
2297    0.0    0.0                9.0  ...        13.0        10.0     5.0

[2298 rows x 23 columns]
<class 'pandas.core.frame.DataFrame'>


In [ ]:
metaData_le[['img_id','lesion_id']]=metaData_pd[['img_id','lesion_id']]

In [ ]:
metaData_le[['diagnostic']] = metaData_pd['diagnostic']

In [ ]:
lesion_type_dict = {
    'ACK': 'Actinic Keratosis',
    'BCC': 'Basal Cell Carcinoma',
    'MEL': 'Melanoma',
    'NEV': 'Nevus',
    'SCC': 'Squamous Cell Carcinoma',
    'SEK': 'Seborrheic Keratosis'
    
}

In [ ]:
skin_image_paths = glob(os.path.join(raw_data_path, '*.png')) #glob patterns specify sets of filenames with wildcard character.

In [ ]:
image_id_and_path_dict = {os.path.basename(x): x for x in skin_image_paths}

In [ ]:
metaData_le['image_path'] = metaData_le['img_id'].map(image_id_and_path_dict.get)  # image path
metaData_le['cell_type'] = metaData_le['diagnostic'].map(lesion_type_dict.get)     # cell type
# metaData_le['cell_type_label'] = pd.Categorical(metaData_pd['cell_type']).codes    # cell type class label

In [48]:
metaData_le.columns

Index(['smoke', 'drink', 'background_father', 'background_mother', 'pesticide',
       'gender', 'skin_cancer_history', 'cancer_history', 'has_piped_water',
       'has_sewage_system', 'region', 'itch', 'grew', 'hurt', 'changed',
       'bleed', 'elevation', 'biopsed', 'age', 'fitspatrick', 'diameter_1',
       'diameter_2', 'target', 'img_id', 'lesion_id', 'diagnostic',
       'image_path', 'cell_type'],
      dtype='object')

**The best predictors we will take for model development are: age, biopsed, grew, region, pesticide, skin_cancer_history, cancer_history, background_father, drink, smoke, itch, bleed, changed, has_sewage_system, elevation**


In [50]:
#dropping all the unwanted columns from the encoded dataframe
#dropping background_mother, gender, has_piped_water, hurt, fitspatrick, diameter1, diameter_2
data_exr=metaData_le.drop(columns=['background_mother','gender','has_piped_water','hurt','diameter_1','diameter_2','target','img_id','lesion_id'])

In [51]:
data_exr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2298 entries, 0 to 2297
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   smoke                2298 non-null   float64
 1   drink                2298 non-null   float64
 2   background_father    2298 non-null   float64
 3   pesticide            2298 non-null   float64
 4   skin_cancer_history  2298 non-null   float64
 5   cancer_history       2298 non-null   float64
 6   has_sewage_system    2298 non-null   float64
 7   region               2298 non-null   float64
 8   itch                 2298 non-null   float64
 9   grew                 2298 non-null   float64
 10  changed              2298 non-null   float64
 11  bleed                2298 non-null   float64
 12  elevation            2298 non-null   float64
 13  biopsed              2298 non-null   float64
 14  age                  2298 non-null   float64
 15  fitspatrick          2298 non-null   f

In [ ]:
data_exr.to_csv("/content/gdrive/My Drive/Data606/Project/Skin_Cancer_Analysis/data/Tablular_Encoded.csv")